Score source: https://es.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_2022-23#Tabla_de_resultados_cruzados
Budget source: https://cronicaglobal.elespanol.com/culemania/palco/20230415/todos-los-presupuestos-de-la-liga/756424401_0.html

In [18]:
import pandas as pd
import numpy as np

def clean_scores(raw_scores):
    team_names = raw_scores.iloc[:, 0].str.strip().copy(deep=True).values
    team_abbrs = raw_scores.columns.str.strip().copy(deep=True).values[1:]
    raw_scores = raw_scores.melt(id_vars=[raw_scores.columns[0]])
    raw_scores.columns = ["home_team", "away_team", "result"]
    raw_scores = raw_scores.apply(lambda x: x.str.strip(), axis="index")
    team_rename_map = {
        'Ath. Club': "Athletic Club",
        'At. Madrid': "Atlético de Madrid",
        'Barcelona': "FC Barcelona",
        'R. Vallecano': "Rayo Vallecano",
        'Celta': "Celta de Vigo",
        'R. Madrid': "Real Madrid",
        'R. Sociedad': "Real Sociedad",
        'R. Valladolid': "Valladolid",
    }
    team_names = [team_rename_map.get(name, name) for name in team_names]
    raw_scores = raw_scores.replace(
        {
            "home_team": team_rename_map,
            "away_team": {abbr: team for abbr, team in zip(team_abbrs, team_names)},
            "result": {"—": np.nan}
        }
    ).dropna(subset=["result"]).reset_index(drop=True)
    df_aux = raw_scores.result.str.split("–", expand=True)
    df_aux.columns = ["home_goals", "away_goals"]
    return pd.concat((raw_scores[["home_team", "away_team"]], df_aux), axis="columns")

In [26]:
raw_scores = pd.read_csv("raw_scores_22-23.tsv", sep="\t")
clean_scores = clean_scores(raw_scores)
clean_scores.to_csv("scores_22-23.csv", index=False)
clean_scores

,home_team,away_team,home_goals,away_goals
0,Athletic Club,Almería,4,0
1,Atlético de Madrid,Almería,2,1
2,Osasuna,Almería,3,1
3,Cádiz,Almería,1,1
4,Elche,Almería,1,1
...,...,...,...,...
375,Mallorca,Villarreal,4,2
376,Real Sociedad,Villarreal,1,0
377,Sevilla,Villarreal,2,1
378,Valencia,Villarreal,1,1
